In [2]:
# Option 1: Upload CoNLL file directly to Colab
from google.colab import files
import os

print("📁 Upload your CoNLL labeled file (e.g., amharic_ecommerce_conll_labeled.txt)")
print("Choose the file from your computer:")

uploaded = files.upload()

# Get the uploaded file name
uploaded_file = list(uploaded.keys())[0]
print(f"✅ File uploaded: {uploaded_file}")

# Optional: Also mount Google Drive if you want to save models there
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Create directory for saving models
os.makedirs('/content/drive/MyDrive/models', exist_ok=True)
print("✅ Google Drive mounted and model directory created")

📁 Upload your CoNLL labeled file (e.g., amharic_ecommerce_conll_labeled.txt)
Choose the file from your computer:


Saving amharic_ecommerce_conll_labeled.txt to amharic_ecommerce_conll_labeled.txt
✅ File uploaded: amharic_ecommerce_conll_labeled.txt
Mounted at /content/drive
✅ Google Drive mounted and model directory created


In [3]:
!pip install evaluate seqeval transformers datasets scikit-learn

from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from evaluate import load
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import pandas as pd

# Improved CoNLL file parser that handles comments and empty lines
def read_conll_file(file_path):
    sentences, labels = [], []
    sentence, label_seq = [], []

    print(f"📖 Reading CoNLL file: {file_path}")

    with open(file_path, encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()

            # Skip empty lines and comments
            if not line or line.startswith('#'):
                if sentence:  # End of sentence
                    sentences.append(sentence)
                    labels.append(label_seq)
                    sentence, label_seq = [], []
                continue

            # Parse token and label
            try:
                parts = line.split()
                if len(parts) >= 2:
                    token = parts[0]
                    tag = parts[1]
                    sentence.append(token)
                    label_seq.append(tag)
                else:
                    print(f"⚠️ Warning: Line {line_num} has unexpected format: '{line}'")
            except Exception as e:
                print(f"❌ Error parsing line {line_num}: '{line}' - {e}")

    # Add final sentence if exists
    if sentence:
        sentences.append(sentence)
        labels.append(label_seq)

    print(f"✅ Parsed {len(sentences)} sentences with {sum(len(s) for s in sentences)} tokens")
    return sentences, labels

# Load the uploaded CoNLL file
try:
    tokens, ner_tags = read_conll_file(uploaded_file)

    # Display sample data
    print(f"\n📊 Dataset Overview:")
    print(f"   • Total sentences: {len(tokens)}")
    print(f"   • Total tokens: {sum(len(s) for s in tokens)}")
    print(f"   • Average tokens per sentence: {sum(len(s) for s in tokens) / len(tokens):.1f}")

    print(f"\n📝 Sample sentences:")
    for i in range(min(3, len(tokens))):
        print(f"   Sentence {i+1}: {' '.join(tokens[i][:10])}...")
        print(f"   Labels {i+1}: {' '.join(ner_tags[i][:10])}...")
        print()

except Exception as e:
    print(f"❌ Error loading CoNLL file: {e}")
    print("Please check that your file is in proper CoNLL format")
    raise

# Create label mappings
unique_tags = sorted(set(tag for seq in ner_tags for tag in seq))
label2id = {label: i for i, label in enumerate(unique_tags)}
id2label = {i: label for label, i in label2id.items()}

print(f"📋 Entity labels found: {unique_tags}")
print(f"📋 Label mappings: {label2id}")

# Convert labels to ID format
ner_ids = [[label2id[tag] for tag in seq] for seq in ner_tags]

# Split into train and test (80-20 split)
train_tokens, test_tokens, train_labels, test_labels = train_test_split(
    tokens, ner_ids, test_size=0.2, random_state=42, stratify=None
)

# Create Hugging Face datasets
dataset_dict = DatasetDict({
    "train": Dataset.from_dict({"tokens": train_tokens, "ner_tags": train_labels}),
    "test": Dataset.from_dict({"tokens": test_tokens, "ner_tags": test_labels})
})

print(f"\n✅ Dataset loaded and split successfully!")
print(f"📊 Train set: {len(train_tokens)} sentences")
print(f"📊 Test set: {len(test_tokens)} sentences")
print(f"📋 Number of unique labels: {len(unique_tags)}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2a04cd62e34f8c247dc654226e3f0a85683213c8e365c148802df9bc87a8b8c0
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
📖 Reading CoNLL file: amharic_ecommerce_conll_labeled.txt
⚠️ Warning: Line 6 has unexpected format: '💯orginal'
⚠️ Warning: Line 11 has unexpected format: '⚡️...'
⚠️ Warning: Line 80 has unexpected format: '...'
⚠️ Warning: Line 157 has unexpected format: '🔰36...'
⚠️ Warning: Line 337 has unexpected format: '🍸500ml'
⚠️ Warning: Line 339 has unexpected format: '...'
✅ Parsed 20 sentences with 657 tokens

📊 Dataset Overview:
   • Total sentences: 20
   • Total tokens: 657
   • Average tokens per sentence: 32.9

📝

In [4]:
# Model selection - Choose the best model for Amharic
model_options = {
    "xlm-roberta": "xlm-roberta-base",           # Good multilingual performance
    "afro-xlmr": "Davlan/afro-xlmr-large",      # Best for African languages including Amharic
    "distilbert": "distilbert-base-multilingual-cased"  # Faster, smaller model
}

# Choose model (afro-xlmr is recommended for Amharic)
model_name = model_options["afro-xlmr"]
print(f"🤖 Using model: {model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
print(f"✅ Tokenizer loaded. Vocab size: {tokenizer.vocab_size}")

# Improved tokenization function with better label alignment
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding=False,  # We'll pad later with data collator
        max_length=512
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens (CLS, SEP, PAD) get -100
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # First token of a word gets the label
                label_ids.append(label[word_idx])
            else:
                # Subsequent tokens of the same word get -100 (ignore in loss)
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply preprocessing to both train and test sets
print("🔄 Tokenizing dataset...")
tokenized_dataset = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names
)

print("✅ Tokenization completed!")
print(f"📊 Tokenized train set: {len(tokenized_dataset['train'])}")
print(f"📊 Tokenized test set: {len(tokenized_dataset['test'])}")

# Display sample tokenized data
sample = tokenized_dataset["train"][0]
print(f"\n📝 Sample tokenized input:")
print(f"   Input IDs length: {len(sample['input_ids'])}")
print(f"   Labels length: {len(sample['labels'])}")
print(f"   First 10 tokens: {tokenizer.convert_ids_to_tokens(sample['input_ids'][:10])}")
print(f"   First 10 labels: {sample['labels'][:10]}")

🤖 Using model: Davlan/afro-xlmr-large


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Tokenizer loaded. Vocab size: 250002
🔄 Tokenizing dataset...


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

✅ Tokenization completed!
📊 Tokenized train set: 16
📊 Tokenized test set: 4

📝 Sample tokenized input:
   Input IDs length: 127
   Labels length: 127
   First 10 tokens: ['<s>', '▁ከ', '45', '▁እስከ', '▁70', '▁መ', 'ለጠ', 'ጥ', '▁የሚችል', '▁ዋጋ']
   First 10 labels: [-100, 2, -100, 7, 2, 7, -100, -100, 7, 7]


In [8]:
# Load model with correct label configuration
print("🤖 Loading pre-trained model...")
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # Handle size mismatches
)

print(f"✅ Model loaded with {len(label2id)} labels")
print(f"📊 Model parameters: {model.num_parameters():,}")

# Optimized training arguments for Colab (Fixed for newer transformers version)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/models/amharic-ner",
    learning_rate=3e-5,           # Slightly higher for fine-tuning
    per_device_train_batch_size=8,   # Reduced to prevent memory issues
    per_device_eval_batch_size=16,   # Reduced batch size for evaluation
    num_train_epochs=3,           # Reduced epochs for faster training
    weight_decay=0.01,
    warmup_steps=50,              # Reduced warmup steps
    logging_steps=10,             # More frequent logging
    eval_strategy="epoch",        # Updated parameter name for newer transformers
    save_strategy="epoch",        # Save after each epoch - MUST match eval_strategy
    save_total_limit=2,           # Keep only best 2 models
    load_best_model_at_end=True,  # Load best model at end
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    push_to_hub=False,            # Don't push to Hugging Face Hub
    report_to="none",             # Disable wandb/tensorboard
    dataloader_pin_memory=False,  # Reduce memory usage
    fp16=True,                    # Enable mixed precision for faster training
    dataloader_num_workers=0,     # Reduce workers for Colab
)

# Load evaluation metric
print("📊 Loading evaluation metrics...")
metric = load("seqeval")
label_list = list(label2id.keys())

print("✅ Training setup completed!")
print(f"📋 Labels to train on: {label_list}")
print(f"🎯 Training will run for {training_args.num_train_epochs} epochs")

🤖 Loading pre-trained model...


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded with 21 labels
📊 Model parameters: 558,862,357
📊 Loading evaluation metrics...


✅ Training setup completed!
📋 Labels to train on: ['2000', 'B-LOC', 'B-PRICE', 'B-Product', 'I-LOC', 'I-PRICE', 'I-Product', 'O', 'moving', 'shape', 'water', 'ላይ', 'ምድጃ', 'ስትሮ', 'ትልቅ', 'አንድ', 'ከፍተኛ', 'ኳሊቲ', 'የፈሳሽ', 'ፊውዝ', 'ፓትራዎች']
🎯 Training will run for 3 epochs


In [9]:
# Enhanced metrics computation function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens) and convert to labels
    true_predictions = []
    true_labels = []

    for prediction, label in zip(predictions, labels):
        true_pred = []
        true_label = []

        for pred_id, label_id in zip(prediction, label):
            if label_id != -100:  # Ignore special tokens
                true_pred.append(id2label[pred_id])
                true_label.append(id2label[label_id])

        if true_pred:  # Only add non-empty sequences
            true_predictions.append(true_pred)
            true_labels.append(true_label)

    # Compute metrics using seqeval
    results = metric.compute(predictions=true_predictions, references=true_labels)

    # Extract per-entity metrics
    detailed_results = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

    # Add per-entity scores if available
    if "per_type" in results:
        for entity_type, scores in results["per_type"].items():
            detailed_results[f"{entity_type}_f1"] = scores["f1"]
            detailed_results[f"{entity_type}_precision"] = scores["precision"]
            detailed_results[f"{entity_type}_recall"] = scores["recall"]

    return detailed_results

# Create data collator for dynamic padding
data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,
    pad_to_multiple_of=8,  # Optimize for GPU
    return_tensors="pt"
)

# Create Trainer instance
print("🏋️ Setting up trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ Trainer ready!")
print(f"📊 Training samples: {len(tokenized_dataset['train'])}")
print(f"📊 Evaluation samples: {len(tokenized_dataset['test'])}")
print("\n🚀 Ready to start training! Run the next cell to begin.")

🏋️ Setting up trainer...


/tmp/ipython-input-9-2000893228.py:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


✅ Trainer ready!
📊 Training samples: 16
📊 Evaluation samples: 4

🚀 Ready to start training! Run the next cell to begin.


In [10]:
# Start training with progress monitoring
print("🚀 Starting fine-tuning...")
print("⏱️ This may take 15-30 minutes depending on your data size and GPU")
print("📊 Training progress will be displayed below:")

try:
    # Train the model
    training_result = trainer.train()

    print("\n🎉 Training completed successfully!")
    print(f"📈 Final training loss: {training_result.training_loss:.4f}")
    print(f"⏱️ Training time: {training_result.metrics['train_runtime']:.2f} seconds")
    print(f"🔄 Total training steps: {training_result.global_step}")

except Exception as e:
    print(f"❌ Training failed with error: {e}")
    print("💡 Tips to fix:")
    print("   • Reduce batch size if you get CUDA out of memory")
    print("   • Check that your CoNLL file format is correct")
    print("   • Ensure you have sufficient disk space for model checkpoints")
    raise

🚀 Starting fine-tuning...
⏱️ This may take 15-30 minutes depending on your data size and GPU
📊 Training progress will be displayed below:


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2696: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,3.317253,0.000000,0.000000,0.000000,0.040000
2,No log,3.317253,0.000000,0.000000,0.000000,0.040000
3,No log,3.317253,0.000000,0.000000,0.000000,0.040000


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ምድጃ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: shape seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2000 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ፊውዝ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ትልቅ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: 


🎉 Training completed successfully!
📈 Final training loss: 3.3350
⏱️ Training time: 155.21 seconds
🔄 Total training steps: 6


In [11]:
# Comprehensive model evaluation
print("📊 Evaluating fine-tuned model...")

# Run evaluation
eval_results = trainer.evaluate(
    eval_dataset=tokenized_dataset["test"],
    metric_key_prefix="test"
)

# Display results
print("\n🎯 EVALUATION RESULTS:")
print("=" * 50)
print(f"📈 Overall F1 Score: {eval_results['test_f1']:.4f}")
print(f"📈 Overall Precision: {eval_results['test_precision']:.4f}")
print(f"📈 Overall Recall: {eval_results['test_recall']:.4f}")
print(f"📈 Overall Accuracy: {eval_results['test_accuracy']:.4f}")

# Display per-entity results if available
entity_metrics = {}
for key, value in eval_results.items():
    if "_f1" in key and key != "test_f1":
        entity_type = key.replace("test_", "").replace("_f1", "")
        entity_metrics[entity_type] = {
            'f1': value,
            'precision': eval_results.get(f"test_{entity_type}_precision", 0),
            'recall': eval_results.get(f"test_{entity_type}_recall", 0)
        }

if entity_metrics:
    print("\n📋 PER-ENTITY PERFORMANCE:")
    print("-" * 50)
    for entity, metrics in entity_metrics.items():
        print(f"{entity:15} | F1: {metrics['f1']:.3f} | P: {metrics['precision']:.3f} | R: {metrics['recall']:.3f}")

print("\n" + "=" * 50)

# Test with sample predictions
print("\n🧪 SAMPLE PREDICTIONS:")
sample_text = tokenized_dataset["test"][0]
predictions = trainer.predict([sample_text])
predicted_labels = np.argmax(predictions.predictions[0], axis=1)

print("Sample tokens and predictions:")
tokens = tokenizer.convert_ids_to_tokens(sample_text['input_ids'])
true_labels = sample_text['labels']

for i, (token, true_id, pred_id) in enumerate(zip(tokens, true_labels, predicted_labels)):
    if true_id != -100:  # Skip special tokens
        true_label = id2label[true_id]
        pred_label = id2label[pred_id]
        match = "✅" if true_label == pred_label else "❌"
        print(f"{match} {token:15} | True: {true_label:10} | Pred: {pred_label:10}")
    if i >= 20:  # Show first 20 tokens only
        break

print(f"\n💾 All evaluation results: {eval_results}")

📊 Evaluating fine-tuned model...


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2696: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(



🎯 EVALUATION RESULTS:
📈 Overall F1 Score: 0.0000
📈 Overall Precision: 0.0000
📈 Overall Recall: 0.0000
📈 Overall Accuracy: 0.0400


🧪 SAMPLE PREDICTIONS:


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ምድጃ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: shape seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 2000 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ፊውዝ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ትልቅ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: 

Sample tokens and predictions:
❌ ▁ባለ             | True: ምድጃ        | Pred: የፈሳሽ      

💾 All evaluation results: {'test_loss': 3.3172526359558105, 'test_precision': 0.0, 'test_recall': 0.0, 'test_f1': 0.0, 'test_accuracy': 0.04, 'test_runtime': 0.0728, 'test_samples_per_second': 54.93, 'test_steps_per_second': 13.732, 'epoch': 3.0}


In [12]:
# Save the fine-tuned model
model_save_path = "/content/drive/MyDrive/models/amharic-ner-final"

print("💾 Saving fine-tuned model...")
try:
    # Save model and tokenizer
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    # Save additional metadata
    import json
    metadata = {
        "model_name": model_name,
        "labels": list(label2id.keys()),
        "label2id": label2id,
        "id2label": id2label,
        "num_train_examples": len(tokenized_dataset["train"]),
        "num_test_examples": len(tokenized_dataset["test"]),
        "training_epochs": training_args.num_train_epochs,
        "final_eval_results": eval_results
    }

    with open(f"{model_save_path}/training_metadata.json", "w", encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)

    print(f"✅ Model saved successfully to: {model_save_path}")
    print(f"📁 Files saved:")
    print(f"   • pytorch_model.bin (model weights)")
    print(f"   • config.json (model configuration)")
    print(f"   • tokenizer.json (tokenizer)")
    print(f"   • training_metadata.json (training information)")

except Exception as e:
    print(f"❌ Error saving model: {e}")
    print("💡 Make sure Google Drive is mounted and you have write permissions")

# Test loading the saved model
print("\n🧪 Testing saved model loading...")
try:
    from transformers import pipeline

    # Create NER pipeline with saved model
    ner_pipeline = pipeline(
        "ner",
        model=model_save_path,
        tokenizer=model_save_path,
        aggregation_strategy="simple"
    )

    # Test with sample Amharic text
    test_texts = [
        "ዋጋ 2500 ብር ባለሁለት ምድጃ ስቶቭ በቦሌ አካባቢ",
        "አዲስ አበባ መርካቶ ውስጥ ቲሸርት ይሸጣል",
        "በ 1500 ብር ጫማ ፒያሳ ላይ"
    ]

    print("🎯 Testing with sample texts:")
    for i, text in enumerate(test_texts, 1):
        print(f"\nTest {i}: {text}")
        try:
            entities = ner_pipeline(text)
            if entities:
                for entity in entities:
                    print(f"   🏷️ {entity['word']} -> {entity['entity_group']} (confidence: {entity['score']:.3f})")
            else:
                print("   No entities detected")
        except Exception as e:
            print(f"   ❌ Error: {e}")

    print(f"\n🎉 Model testing completed successfully!")
    print(f"📍 Your fine-tuned Amharic NER model is ready to use!")
    print(f"📁 Model location: {model_save_path}")

except Exception as e:
    print(f"⚠️ Could not test pipeline: {e}")
    print("Model was saved but pipeline testing failed")

print("\n" + "="*60)
print("🎉 TASK 3 COMPLETED SUCCESSFULLY!")
print("✅ Fine-tuned NER model for Amharic e-commerce text")
print("✅ Model saved and tested")
print("✅ Ready for deployment and use")
print("="*60)

💾 Saving fine-tuned model...
✅ Model saved successfully to: /content/drive/MyDrive/models/amharic-ner-final
📁 Files saved:
   • pytorch_model.bin (model weights)
   • config.json (model configuration)
   • tokenizer.json (tokenizer)
   • training_metadata.json (training information)

🧪 Testing saved model loading...


Device set to use cuda:0


🎯 Testing with sample texts:

Test 1: ዋጋ 2500 ብር ባለሁለት ምድጃ ስቶቭ በቦሌ አካባቢ
   🏷️ ዋጋ -> አንድ (confidence: 0.105)
   🏷️ 2500 -> shape (confidence: 0.116)
   🏷️ ብር -> ላይ (confidence: 0.159)
   🏷️ ባለ -> 2000 (confidence: 0.093)
   🏷️ ሁ -> Product (confidence: 0.108)
   🏷️ ለት -> ላይ (confidence: 0.135)
   🏷️ ምድ -> 2000 (confidence: 0.180)
   🏷️ ጃ -> ላይ (confidence: 0.152)
   🏷️  -> 2000 (confidence: 0.126)
   🏷️ ስቶ -> ምድጃ (confidence: 0.111)
   🏷️ ቭ -> አንድ (confidence: 0.123)
   🏷️ በ -> 2000 (confidence: 0.132)
   🏷️ ቦ -> ላይ (confidence: 0.206)
   🏷️ ሌ -> አንድ (confidence: 0.207)
   🏷️ አካባቢ -> 2000 (confidence: 0.236)

Test 2: አዲስ አበባ መርካቶ ውስጥ ቲሸርት ይሸጣል
   🏷️ አዲስ አበባ መርካ -> 2000 (confidence: 0.164)
   🏷️ ውስጥ  -> አንድ (confidence: 0.164)

Test 3: በ 1500 ብር ጫማ ፒያሳ ላይ
   🏷️ በ 1500 -> 2000 (confidence: 0.153)
   🏷️ ብር -> አንድ (confidence: 0.143)
   🏷️ ጫ -> 2000 (confidence: 0.146)
   🏷️ ፒያሳ ላይ -> 2000 (confidence: 0.145)

🎉 Model testing completed successfully!
📍 Your fine-tuned Amharic NER model is re